# Yearly Maxima Return Level Estimation: Example 4

In this notebook, we proceed to the step by step anylsis for the estimation of the 10-year return level of groundwater level using statistical analysis. 

Example 4 described the more complex cases where multiple advanced tools, such as trend estimation and increased block sizes, are necessary to obtain accurate estimates.

We start by loading all the libraries necessary to perform the analysis.

In [1]:
# Load necessary libraries

# Library for easy date manipulation
library(lubridate) 

#Library for estimation of parametric models using l-moment matching
library(lmom)

# Library for automatic trend estimation
library(mgcv)

# Library for data manipulation
library(dplyr)

# Libraries for data plotting
library(ggplot2)
library(repr)


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


Loading required package: nlme

This is mgcv 1.9-0. For overview type 'help("mgcv-package")'.


Attaching package: ‘dplyr’


The following object is masked from ‘package:nlme’:

    collapse


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




We now load and format the data into the R environment.

In [3]:
data <- read.csv("../data/data_groundwater_level_4.csv", header = TRUE, sep = ",")
head(data)

,Zeitstempel,Wert
,<chr>,<dbl>
1,1977-01-01,426.78
2,1977-01-02,426.78
3,1977-01-03,426.78
4,1977-01-04,426.78
5,1977-01-05,426.78
6,1977-01-06,426.78
